<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-04 14:35:28
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.39 C
-------------------
Today PnL: 51.71 K (0.37%)
Current PnL: -30.00 L (-19.13%)
CY Booked + Current PnL: -15.26 L (-9.73%)
-------------------
Total profit:  1.36 L
Total loss:  -31.36 L
-------------------
Total Booked + Current PnL: 11.72 L (9.03%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.57%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 95.43 L (68.41%)
Deployed:  1.30 C
Current:  1.39 C
CAGR/XIRR %: 3.76%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,1.36,7.03,11.39,19.23,18343.0,10582.0,161042.0,484.83,25.73,56.0,L-LC,12.66,182.0,0.58,1.15,29.81,XY25,ATH,MINING
50,MASFIN,-1.73,-4.59,27.42,21.57,25632.0,-4500.0,93480.0,397.04,-17.80,49.0,H-SC,3.45,168.0,-0.18,0.67,36.42,XR,ATH,FINANCE
77,TTKPRESTIG,-0.96,-27.89,38.81,0.09,28202.0,-28111.0,72666.0,770.00,68.62,33.0,M-SC,7.62,253.0,-1.00,0.52,0.00,OX40N,NTT,DURABLES
57,RECLTD,2.59,3.45,16.71,20.74,35121.0,7012.0,210182.0,446.00,35.83,60.0,M-MC,4.39,189.0,0.20,1.51,14.43,XY25,NTT,FINANCE
51,MEDANTA,0.78,-6.32,32.45,24.08,38590.0,-8029.0,118921.0,1486.00,-13.83,50.0,X-SC,10.66,91.0,-0.21,0.85,8.92,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SFL,17.01,-39.72,110.07,26.63,173981.0,-104154.0,158064.0,1287.00,19.71,66.0,M-SC,16.40,241.0,-0.60,1.13,21.33,XY24,NTT,MISC
23,DIXON,6.09,-18.86,61.77,31.26,115663.0,-43520.0,187248.0,18931.72,-62.08,56.0,X-MC,27.69,56.0,-0.38,1.34,14.92,X40N,ATH,IT
55,RAJESHEXPO,5.00,-65.62,191.06,0.07,90108.0,-90015.0,47162.0,518.00,1628.54,55.0,L-SC,12.01,269.0,-1.00,0.34,17.29,OX40N,NTT,JEWELLERY
82,VOLTAS,3.99,11.20,30.57,45.20,65183.0,21483.0,213225.0,1856.00,0.30,57.0,X-MC,3.12,80.0,0.33,1.53,18.76,XY25,NTT,AC
5,ANGELONE,3.92,10.13,12.78,24.20,42962.0,30912.0,336162.0,3033.00,56.91,59.0,X-SC,1.45,99.0,0.72,2.41,40.79,X40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-7.31,4.50,28.47,34.24,93084.0,14067.0,326955.0,1972.00,-18.52,34.0,X-LC,6.89,6.0,0.15,2.34,13.15,X40,NTT,IT
73,TCS,-6.93,-17.46,43.63,18.55,136207.0,-66050.0,312187.0,4311.59,-28.80,38.0,X-LC,0.49,3.0,-0.48,2.24,6.21,X40,ATH,IT
30,HAPPSTMNDS,-5.05,-50.35,272.27,84.85,173273.0,-64525.0,63640.0,1480.71,-41.05,32.0,H-SC,22.70,153.0,-0.37,0.46,0.00,AR,ATH,IT
33,HCLTECH,-4.32,5.26,17.64,23.83,44921.0,12730.0,254654.0,1908.19,9.79,40.0,X-LC,7.62,13.0,0.28,1.83,22.18,X40,ATH,IT
84,WIPRO,-3.95,-4.39,80.17,72.27,129884.0,-7437.0,162011.0,420.00,-17.88,38.0,M-LC,2.54,101.0,-0.06,1.16,4.14,XR,NTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,0.81,-5.65,120.36,107.91,167669.0,-8338.0,139306.0,1641.55,-23.39,50.0,M-SC,11.39,220.0,-0.05,1.00,4.80,OX40N,ATH,CABLES
66,SIS,1.77,-20.08,51.72,21.26,45896.0,-22292.0,88740.0,528.00,2098.96,64.0,H-SC,4.78,163.0,-0.49,0.64,19.90,OX40N,NTT,MISC
72,TATAELXSI,-0.24,-20.81,66.72,32.03,69659.0,-27430.0,104405.0,9161.00,-14.36,54.0,H-SC,6.46,157.0,-0.39,0.75,16.94,OX40N,NTT,IT
48,KANSAINER,-2.54,-25.80,52.92,13.47,105896.0,-69561.0,200106.0,340.00,-69.48,40.0,H-SC,5.21,159.0,-0.66,1.43,1.75,XY24,NTT,PAINTS
40,INDIGOPNTS,-0.64,-27.31,37.59,0.02,47698.0,-47670.0,126889.0,1408.00,93.64,30.0,M-SC,4.78,240.0,-1.00,0.91,10.37,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,2.59,3.45,16.71,20.74,35121.0,7012.0,210182.0,446.00,35.83,60.0,M-MC,4.39,189.0,0.20,1.51,14.43,XY25,NTT,FINANCE
18,COALINDIA,1.36,7.03,11.39,19.23,18343.0,10582.0,161042.0,484.83,25.73,56.0,L-LC,12.66,182.0,0.58,1.15,29.81,XY25,ATH,MINING
1,ABB,1.21,5.26,38.30,45.57,105469.0,13757.0,275376.0,7934.00,-32.38,71.0,H-MC,5.63,121.0,0.13,1.97,22.28,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,1.21,5.26,38.30,45.57,105469.0,13757.0,275376.0,7934.00,-32.38,71.0,H-MC,5.63,121.0,0.13,1.97,22.28,AR,NTT,ELECTRICAL
39,INDIAMART,0.40,-2.17,111.31,106.72,134307.0,-2676.0,120660.0,4810.62,-51.58,57.0,H-SC,8.68,138.0,-0.02,0.87,20.70,AR,ATH,MISC
26,FINCABLES,0.81,-5.65,120.36,107.91,167669.0,-8338.0,139306.0,1641.55,-23.39,50.0,M-SC,11.39,220.0,-0.05,1.00,4.80,OX40N,ATH,CABLES
84,WIPRO,-3.95,-4.39,80.17,72.27,129884.0,-7437.0,162011.0,420.00,-17.88,38.0,M-LC,2.54,101.0,-0.06,1.16,4.14,XR,NTT,IT
85,ZYDUSLIFE,0.03,-4.76,42.41,35.63,84649.0,-9978.0,199596.0,1286.17,-18.07,53.0,H-MC,5.96,119.0,-0.12,1.43,11.12,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-0.77,-44.52,143.68,35.20,102576.0,-57278.0,71392.0,2096.00,-6.43,27.0,X-SC,20.67,93.0,-0.56,0.51,1.46,X40,NTT,FOOTWEAR
44,ITC,1.26,-23.03,43.93,10.78,81260.0,-55343.0,184975.0,452.00,-54.23,30.0,X-LC,18.51,1.0,-0.68,1.33,1.88,X40,NTT,FMCG
81,VBL,-1.23,-10.00,50.74,35.67,144234.0,-31581.0,284261.0,671.64,-20.77,30.0,X-LC,0.68,5.0,-0.22,2.04,2.47,X40N,ATH,FMCG
6,ASIANPAINT,1.35,-14.06,40.73,20.95,88126.0,-35402.0,216366.0,3460.25,-14.97,31.0,X-LC,9.89,36.0,-0.40,1.55,16.56,X40,ATH,PAINTS
42,INFY,-7.31,4.50,28.47,34.24,93084.0,14067.0,326955.0,1972.00,-18.52,34.0,X-LC,6.89,6.0,0.15,2.34,13.15,X40,NTT,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,-6.93,-17.46,43.63,18.55,136207.0,-66050.0,312187.0,4311.59,-28.80,38.0,X-LC,0.49,3.0,-0.48,2.24,6.21,X40,ATH,IT
81,VBL,-1.23,-10.00,50.74,35.67,144234.0,-31581.0,284261.0,671.64,-20.77,30.0,X-LC,0.68,5.0,-0.22,2.04,2.47,X40N,ATH,FMCG
5,ANGELONE,3.92,10.13,12.78,24.20,42962.0,30912.0,336162.0,3033.00,56.91,59.0,X-SC,1.45,99.0,0.72,2.41,40.79,X40N,NTT,FINANCE
34,HINDUNILVR,0.09,-7.77,23.25,13.67,55121.0,-19983.0,237080.0,2922.00,-14.61,50.0,X-LC,2.31,9.0,-0.36,1.70,11.92,XY25,NTT,FMCG
21,DABUR,0.44,-1.72,46.21,43.70,112665.0,-4258.0,243810.0,735.00,-9.79,44.0,X-MC,2.57,73.0,-0.04,1.75,13.08,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-0.77,-44.52,143.68,35.20,102576.0,-57278.0,71392.0,2096.00,-6.43,27.0,X-SC,20.67,93.0,-0.56,0.51,1.46,X40,NTT,FOOTWEAR
44,ITC,1.26,-23.03,43.93,10.78,81260.0,-55343.0,184975.0,452.00,-54.23,30.0,X-LC,18.51,1.0,-0.68,1.33,1.88,X40,NTT,FMCG
53,PGHH,0.88,-9.82,48.81,34.19,93981.0,-20976.0,192544.0,17907.65,-36.54,41.0,X-MC,8.30,60.0,-0.22,1.38,2.38,X40,ATH,FMCG
81,VBL,-1.23,-10.00,50.74,35.67,144234.0,-31581.0,284261.0,671.64,-20.77,30.0,X-LC,0.68,5.0,-0.22,2.04,2.47,X40N,ATH,FMCG
10,BAJAJHLDNG,-0.16,-2.56,32.61,29.22,60415.0,-4859.0,185266.0,14451.52,-5.33,51.0,X-LC,14.13,29.0,-0.08,1.33,3.20,X40,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,2.30,-27.13,96.02,42.84,45485.0,-17636.0,47370.0,424.00,-52.07,60.0,X-SC,21.42,83.0,-0.39,0.34,11.35,XY24,NTT,MISC
12,BATAINDIA,-0.77,-44.52,143.68,35.20,102576.0,-57278.0,71392.0,2096.00,-6.43,27.0,X-SC,20.67,93.0,-0.56,0.51,1.46,X40,NTT,FOOTWEAR
58,RELAXO,-1.63,-51.41,213.47,52.31,150556.0,-74632.0,70528.0,1176.00,-48.31,44.0,X-SC,12.01,92.0,-0.50,0.51,4.88,X40N,NTT,FOOTWEAR
51,MEDANTA,0.78,-6.32,32.45,24.08,38590.0,-8029.0,118921.0,1486.00,-13.83,50.0,X-SC,10.66,91.0,-0.21,0.85,8.92,XY24,NTT,HEALTHCARE
35,HONAUT,-2.31,-18.82,76.55,43.32,101214.0,-30652.0,132220.0,58357.33,-30.28,49.0,X-SC,8.97,90.0,-0.30,0.95,7.36,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,1.26,-23.03,43.93,10.78,81260.0,-55343.0,184975.0,452.00,-54.23,30.0,X-LC,18.51,1.0,-0.68,1.33,1.88,X40,NTT,FMCG
75,TMPV,0.55,-23.32,60.38,22.98,104583.0,-52690.0,173208.0,600.00,-20.63,65.0,X-LC,4.41,2.0,-0.50,1.24,10.73,XY24,NTT,AUTO
73,TCS,-6.93,-17.46,43.63,18.55,136207.0,-66050.0,312187.0,4311.59,-28.80,38.0,X-LC,0.49,3.0,-0.48,2.24,6.21,X40,ATH,IT
81,VBL,-1.23,-10.00,50.74,35.67,144234.0,-31581.0,284261.0,671.64,-20.77,30.0,X-LC,0.68,5.0,-0.22,2.04,2.47,X40N,ATH,FMCG
42,INFY,-7.31,4.50,28.47,34.24,93084.0,14067.0,326955.0,1972.00,-18.52,34.0,X-LC,6.89,6.0,0.15,2.34,13.15,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,0.05,-31.87,95.05,32.89,49998.0,-24604.0,52602.0,1800.00,-749.97,60.0,L-MC,11.27,259.0,-0.49,0.38,44.86,XR,NTT,BANKS
67,SONACOMS,0.53,-7.79,50.85,39.09,47430.0,-7884.0,93275.0,804.02,-27.83,74.0,M-SC,1.86,227.0,-0.17,0.67,31.95,AR,ATH,AUTO
14,BSOFT,-2.13,-20.61,89.93,50.79,100419.0,-28989.0,111664.0,831.70,1.66,57.0,H-SC,6.43,171.0,-0.29,0.80,30.01,XR,ATH,IT
18,COALINDIA,1.36,7.03,11.39,19.23,18343.0,10582.0,161042.0,484.83,25.73,56.0,L-LC,12.66,182.0,0.58,1.15,29.81,XY25,ATH,MINING
65,SIEMENS,2.86,-11.24,41.41,25.51,68401.0,-20915.0,165180.0,4671.50,1.35,69.0,H-LC,3.79,50.0,-0.31,1.18,21.72,AR,ATH,ELECTRICAL


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,3.92,10.13,12.78,24.20,42962.0,30912.0,336162.0,3033.00,56.91,59.0,X-SC,1.45,99.0,0.72,2.41,40.79,X40N,NTT,FINANCE
67,SONACOMS,0.53,-7.79,50.85,39.09,47430.0,-7884.0,93275.0,804.02,-27.83,74.0,M-SC,1.86,227.0,-0.17,0.67,31.95,AR,ATH,AUTO
1,ABB,1.21,5.26,38.30,45.57,105469.0,13757.0,275376.0,7934.00,-32.38,71.0,H-MC,5.63,121.0,0.13,1.97,22.28,AR,NTT,ELECTRICAL
76,TRIDENT,2.09,-21.53,63.77,28.51,46167.0,-19858.0,72396.0,48.00,-7.08,69.0,M-SC,10.28,226.0,-0.43,0.52,23.88,XR,NTT,TEXTILES
62,SATIN,0.97,-12.20,72.94,51.83,176932.0,-33712.0,242572.0,274.00,-21.16,63.0,H-SC,3.05,148.0,-0.19,1.74,18.17,XY24,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.38
1,25,44.59
2,50,76.66


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.46
MC    30.80
LC    23.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.60
X40      23.38
X40N     15.13
AR        9.04
XY25      8.75
XR        8.74
OX40N     7.43
SR        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.05
H-SC    23.00
X-LC    20.24
M-SC    11.30
X-SC    10.39
H-MC     4.86
M-MC     1.51
H-LC     1.18
M-LC     1.16
L-LC     1.15
L-SC     0.77
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.89,-14.08,52.64
FINANCE,13.53,-10.68,54.24
IT,12.45,-23.17,84.70
MISC,7.22,-29.11,81.93
ELECTRICAL,6.12,-9.75,49.80
PAINTS,5.28,-25.15,43.74
INSURANCE,4.79,-1.56,36.31
PHARMA,3.97,-6.05,39.43
AUTO,2.97,-26.65,70.45


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3258414.0,21
AR,1309869.0,10
XR,1272005.0,12
X40,1241244.0,15
X40N,1013728.0,10
OX40N,755895.0,10
XY25,398578.0,6
SR,293153.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3586892.0,24
X-MC,1677826.0,16
M-SC,1481674.0,15
X-LC,1068395.0,12
X-SC,842136.0,9
H-MC,400605.0,3
L-SC,183611.0,2
M-LC,129884.0,1
H-LC,68401.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1226844.0      6
           AR         930337.0      5
M-SC       XY24       832812.0      6
H-SC       XR         816429.0      7
X-MC       X40        565866.0      7
X-LC       X40        504013.0      6
X-MC       XY24       475856.0      3
           X40N       410448.0      4
X-SC       X40N       365245.0      4
M-SC       OX40N      345658.0      5
H-SC       OX40N      320129.0      4
X-SC       XY24       305526.0      3
H-SC       SR         293153.0      2
X-LC       X40N       238035.0      2
X-MC       XY25       225656.0      2
H-MC       XY24       210487.0      1
X-LC       XY24       206889.0      2
H-MC       AR         190118.0      2
M-SC       XR         182191.0      2
X-SC       X40        171365.0      2
M-LC       XR         129884.0      1
M-SC       AR         121013.0      2
X-LC       XY25       119458.0      2
L-SC       XR          93503.0      1
           OX40N       90108.0      1
H-LC       AR          68401.0      1
L-MC       XR          49998.0      1
M-MC       XY25        35121.0      1
L-LC       XY25        18343.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 23.0 seconds
